In [760]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [761]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, make_pipeline, Pipeline

In [762]:
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

In [763]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


In [764]:
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable


In [765]:
import xgboost as xgb
import catboost as cat
import lightgbm as light

In [766]:
! pip install scikit-multilearn

Defaulting to user installation because normal site-packages is not writeable


In [767]:
!pip install community

Defaulting to user installation because normal site-packages is not writeable


In [768]:
!pip install networkx

Defaulting to user installation because normal site-packages is not writeable


In [769]:
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from skmultilearn.cluster.networkx import NetworkXLabelGraphClusterer
from skmultilearn.cluster import LabelCooccurrenceGraphBuilder
from skmultilearn.ensemble import LabelSpacePartitioningClassifier

> import datasets

In [770]:
test_path = '../datasets/Test.csv'
train_path = '../datasets/Train.csv'
sample_submission_path = '../datasets/SampleSubmission.csv'

> define utitlity functions

load data

In [771]:
def load_data(path=""):
    data = pd.read_csv(path)
    return data

move columns

In [772]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()    
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

transform dates

In [773]:
def transform_dates(df, col_name='', calc_age=False, 
                    birth_year_col_name = '', drop_date=True):
    df[col_name] = pd.to_datetime(df[col_name])
    #fill empty date columns
    if df[col_name].isna().any() == True:
        fill_date = train_set.join_date.mode()[0]
        df[col_name] =  df[col_name].fillna(fill_date)
    
    #create new datelike features
    todays_date = date.today()
    todays_date = pd.to_datetime(todays_date)
    
    df['duration_till_date'] = (todays_date - df[col_name]).astype('str').apply(lambda x: x.split(' d'
                                                                             )[0].strip()).astype('int64')
    
    df['year'] = df[col_name].apply(lambda x: x.year)
    df['month'] = df[col_name].apply(lambda x: x.year)
    df['week_of_year'] = df[col_name].apply(lambda x: x.weekofyear)
    df['day_of_week'] = df[col_name].apply(lambda x: x.dayofweek)
    
    if calc_age:
        
        df['age'] = df['year'] - df[birth_year_col_name]
        
    
    if drop_date:
        df.drop(columns=[col_name], inplace=True)
        
    
    return df

In [774]:
def val_counts(df, columns=[]):
    for col in columns:
        print("**"*8)
        print(col)
        print("**"*8) 
        print(df[col].value_counts()) 

In [775]:
test_set = load_data(test_path)
train_set = load_data(train_path)
sample_submission = load_data(sample_submission_path)

In [776]:
train_set

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,1/8/2019,M,M,1990,1X1H,BP09,56SI,0,0,...,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,1/8/2019,M,M,1990,748L,NO3L,T4MS,0,0,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29127,8SBA93U,1/9/2019,F,M,1974,748L,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
29128,Y73ETTO,1/11/2018,M,M,1987,XX25,Q6J6,90QI,0,0,...,0,0,0,1,0,0,0,0,0,0
29129,MVXG2LC,1/2/2019,M,M,1985,BOAS,6PE7,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
29130,BUQ8DQN,1/7/2019,M,M,1989,UAOD,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0


In [777]:
test_set

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,F86J5PC,1/12/2018,M,M,1984,94KC,DZRV,90QI,0,0,...,0,0,0,0,0,0,0,0,0,0
1,H6141K3,1/10/2019,M,M,1996,1X1H,J9SY,90QI,0,0,...,0,0,0,1,0,0,0,0,0,0
2,RBAYUXZ,1/1/2020,F,W,1968,UAOD,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
3,KCBILBQ,1/2/2019,M,M,1989,94KC,2A7I,T4MS,0,0,...,0,0,0,0,0,0,0,0,0,0
4,LSEC1ZJ,1/2/2020,F,M,1982,UAOD,0KID,T4MS,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0GMU5UH,1/4/2019,F,M,1986,748L,2A7I,T4MS,0,0,...,0,0,0,0,1,0,1,0,0,0
9996,I9W11CD,1/8/2019,M,M,1980,UAOD,0KID,T4MS,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,42WTEGT,1/4/2018,M,M,1993,748L,SST3,56SI,0,0,...,0,0,0,1,0,0,0,0,0,0
9998,8EKC4O9,1/5/2015,F,U,1981,49BM,2A7I,T4MS,0,0,...,0,0,0,0,0,0,0,0,0,0


In [778]:
sample_submission

,ID X PCODE,Label
0,F86J5PC X P5DA,0
1,F86J5PC X RIBP,0
2,F86J5PC X 8NN1,0
3,F86J5PC X 7POT,0
4,F86J5PC X 66FJ,0
...,...,...
209995,WHII93K X JWFN,0
209996,WHII93K X JZ9D,0
209997,WHII93K X J9JW,0
209998,WHII93K X GHYX,0


### Data Cleaning

In [779]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29132 entries, 0 to 29131
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID                        29132 non-null  object
 1   join_date                 29130 non-null  object
 2   sex                       29132 non-null  object
 3   marital_status            29132 non-null  object
 4   birth_year                29132 non-null  int64 
 5   branch_code               29132 non-null  object
 6   occupation_code           29132 non-null  object
 7   occupation_category_code  29132 non-null  object
 8   P5DA                      29132 non-null  int64 
 9   RIBP                      29132 non-null  int64 
 10  8NN1                      29132 non-null  int64 
 11  7POT                      29132 non-null  int64 
 12  66FJ                      29132 non-null  int64 
 13  GYSR                      29132 non-null  int64 
 14  SOP4                  

> Check values in columns

In [780]:
val_counts(train_set, columns=train_set.columns.to_list())

****************
ID
****************
9EGDEN0    1
A3ZGLU7    1
TPG6MBW    1
B46SPHV    1
22MBOJC    1
          ..
QUQ44D9    1
6LJUIN9    1
05ELFH6    1
ZQD2GSI    1
GXCN6SD    1
Name: ID, Length: 29132, dtype: int64
****************
join_date
****************
1/5/2018      1319
1/5/2019      1199
1/4/2018      1123
1/4/2019      1043
1/1/2019       754
              ... 
22/10/2012       1
11/1/2013        1
7/9/2012         1
8/9/2012         1
23/5/2013        1
Name: join_date, Length: 132, dtype: int64
****************
sex
****************
M    20329
F     8803
Name: sex, dtype: int64
****************
marital_status
****************
M    22913
S     3267
U     2474
W      301
D      135
R       34
P        7
f        1
Name: marital_status, dtype: int64
****************
birth_year
****************
1983    1263
1984    1241
1982    1222
1980    1143
1986    1097
        ... 
1932       1
2011       1
1946       1
2005       1
1936       1
Name: birth_year, Length: 61, dtype: int64

> For every product codes, there is an `imbalance in the class`. lesser product patronage among the class

In [781]:
train_set.isna().any()

ID                          False
join_date                    True
sex                         False
marital_status              False
birth_year                  False
branch_code                 False
occupation_code             False
occupation_category_code    False
P5DA                        False
RIBP                        False
8NN1                        False
7POT                        False
66FJ                        False
GYSR                        False
SOP4                        False
RVSZ                        False
PYUQ                        False
LJR9                        False
N2MW                        False
AHXO                        False
BSTQ                        False
FM3X                        False
K6QO                        False
QBOL                        False
JWFN                        False
JZ9D                        False
J9JW                        False
GHYX                        False
ECY3                        False
dtype: bool

date as empty rows

no more null values in date

### Feature Enngineering

> feature engineer dates and time

In [782]:
train_set = transform_dates(train_set, col_name='join_date', calc_age=True, 
                            birth_year_col_name='birth_year', drop_date=True) 

In [783]:
train_set

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,...,JZ9D,J9JW,GHYX,ECY3,duration_till_date,year,month,week_of_year,day_of_week,age
0,4WKQSBB,F,M,1987,1X1H,2A7I,T4MS,0,0,0,...,0,0,0,0,611,2019,2019,1,2,32
1,CP5S02H,F,M,1981,UAOD,2A7I,T4MS,0,0,0,...,0,0,0,0,607,2019,2019,1,6,38
2,2YKDILJ,M,U,1991,748L,QZYX,90QI,0,0,0,...,0,0,0,1,2798,2013,2013,1,6,22
3,2S9E81J,M,M,1990,1X1H,BP09,56SI,0,0,0,...,0,0,0,0,605,2019,2019,2,1,29
4,BHDYVFT,M,M,1990,748L,NO3L,T4MS,0,0,0,...,1,1,0,0,605,2019,2019,2,1,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29127,8SBA93U,F,M,1974,748L,2A7I,T4MS,0,0,0,...,0,0,0,0,604,2019,2019,2,2,45
29128,Y73ETTO,M,M,1987,XX25,Q6J6,90QI,0,0,0,...,0,0,0,0,967,2018,2018,2,3,31
29129,MVXG2LC,M,M,1985,BOAS,6PE7,T4MS,0,0,0,...,0,0,0,0,611,2019,2019,1,2,34
29130,BUQ8DQN,M,M,1989,UAOD,2A7I,T4MS,0,0,0,...,0,0,0,0,606,2019,2019,2,0,30


In [784]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29132 entries, 0 to 29131
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID                        29132 non-null  object
 1   sex                       29132 non-null  object
 2   marital_status            29132 non-null  object
 3   birth_year                29132 non-null  int64 
 4   branch_code               29132 non-null  object
 5   occupation_code           29132 non-null  object
 6   occupation_category_code  29132 non-null  object
 7   P5DA                      29132 non-null  int64 
 8   RIBP                      29132 non-null  int64 
 9   8NN1                      29132 non-null  int64 
 10  7POT                      29132 non-null  int64 
 11  66FJ                      29132 non-null  int64 
 12  GYSR                      29132 non-null  int64 
 13  SOP4                      29132 non-null  int64 
 14  RVSZ                  

> reorder columns

In [785]:
train_set = movecol(train_set, ['age','duration_till_date', 'year', 'month', 
                                'week_of_year', 'day_of_week'], ref_col='ID')

In [786]:
train_set.head()

,ID,age,duration_till_date,year,month,week_of_year,day_of_week,sex,marital_status,birth_year,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,32,611,2019,2019,1,2,F,M,1987,...,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,38,607,2019,2019,1,6,F,M,1981,...,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,22,2798,2013,2013,1,6,M,U,1991,...,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,29,605,2019,2019,2,1,M,M,1990,...,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,29,605,2019,2019,2,1,M,M,1990,...,0,0,0,0,0,0,1,1,0,0


In [787]:
train_set.columns

Index(['ID', 'age', 'duration_till_date', 'year', 'month', 'week_of_year',
       'day_of_week', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3'],
      dtype='object')

> Melt product IDs to single columns

In [788]:
product_ids  = train_set.columns.to_list()[13:]
product_ids

['P5DA',
 'RIBP',
 '8NN1',
 '7POT',
 '66FJ',
 'GYSR',
 'SOP4',
 'RVSZ',
 'PYUQ',
 'LJR9',
 'N2MW',
 'AHXO',
 'BSTQ',
 'FM3X',
 'K6QO',
 'QBOL',
 'JWFN',
 'JZ9D',
 'J9JW',
 'GHYX',
 'ECY3']

In [789]:
f'lenght of insurance products {len(product_ids)}'

'lenght of insurance products 21'

In [790]:
non_product_ids = train_set.iloc[:, :12].columns.tolist()

In [791]:
non_product_ids

['ID',
 'age',
 'duration_till_date',
 'year',
 'month',
 'week_of_year',
 'day_of_week',
 'sex',
 'marital_status',
 'birth_year',
 'branch_code',
 'occupation_code']

In [792]:
train_set=train_set.melt(id_vars=non_product_ids, value_vars=product_ids, var_name='product_ids', value_name='labels')

In [793]:
train_set

,ID,age,duration_till_date,year,month,week_of_year,day_of_week,sex,marital_status,birth_year,branch_code,occupation_code,product_ids,labels
0,4WKQSBB,32,611,2019,2019,1,2,F,M,1987,1X1H,2A7I,P5DA,0
1,CP5S02H,38,607,2019,2019,1,6,F,M,1981,UAOD,2A7I,P5DA,0
2,2YKDILJ,22,2798,2013,2013,1,6,M,U,1991,748L,QZYX,P5DA,0
3,2S9E81J,29,605,2019,2019,2,1,M,M,1990,1X1H,BP09,P5DA,0
4,BHDYVFT,29,605,2019,2019,2,1,M,M,1990,748L,NO3L,P5DA,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611767,8SBA93U,45,604,2019,2019,2,2,F,M,1974,748L,2A7I,ECY3,0
611768,Y73ETTO,31,967,2018,2018,2,3,M,M,1987,XX25,Q6J6,ECY3,0
611769,MVXG2LC,34,611,2019,2019,1,2,M,M,1985,BOAS,6PE7,ECY3,0
611770,BUQ8DQN,30,606,2019,2019,2,0,M,M,1989,UAOD,2A7I,ECY3,0


> sort dataset by customer ID and product id

In [794]:
train_set.sort_values(by=['ID','product_ids' ], inplace=True)

In [795]:
train_set

,ID,age,duration_till_date,year,month,week_of_year,day_of_week,sex,marital_status,birth_year,branch_code,occupation_code,product_ids,labels
135107,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,66FJ,0
105975,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,7POT,0
76843,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,8NN1,0
339031,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,AHXO,0
368163,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,BSTQ,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260571,ZZZU5BE,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,PYUQ,0
464495,ZZZU5BE,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,QBOL,0
56647,ZZZU5BE,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,RIBP,0
231439,ZZZU5BE,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,RVSZ,1


<!-- > concatenate product ID and customer ID -->

In [796]:
val_counts(train_set, train_set.columns.to_list())

****************
ID
****************
9EGDEN0    21
7MNKSID    21
MO6BP6W    21
41CHTND    21
23Z5Q32    21
           ..
H1ARWUU    21
982SCPC    21
4R3ZLXO    21
S47UIC6    21
GXCN6SD    21
Name: ID, Length: 29132, dtype: int64
****************
age
****************
36    24927
35    24318
34    24087
33    23751
37    23730
39    22638
38    22407
32    22176
31    21903
30    21819
40    21651
41    20412
29    20244
44    20223
42    20097
43    18669
45    18648
46    17661
28    17052
27    16212
47    16086
48    15918
26    14910
49    14658
25    12726
50    12474
51    11928
24    11403
52     9408
23     8757
53     7896
22     6930
54     6300
55     6216
56     5607
59     4809
58     4557
57     4494
21     4473
20     3234
60     2814
19     1512
61      483
18      420
63      357
62      189
17      105
65      105
64      105
66      105
15       21
80       21
76       21
67       21
70       21
72       21
74       21
8        21
Name: age, dtype: int64
*************

> Find ratio of class

In [797]:
train_set.labels.value_counts()

0    545419
1     66353
Name: labels, dtype: int64

The total number of subscriptions is more than no subscriptions -> imbalanced class

> Further Feature Engineering

In [798]:
train_set

,ID,age,duration_till_date,year,month,week_of_year,day_of_week,sex,marital_status,birth_year,branch_code,occupation_code,product_ids,labels
135107,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,66FJ,0
105975,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,7POT,0
76843,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,8NN1,0
339031,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,AHXO,0
368163,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,BSTQ,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260571,ZZZU5BE,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,PYUQ,0
464495,ZZZU5BE,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,QBOL,0
56647,ZZZU5BE,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,RIBP,0
231439,ZZZU5BE,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,RVSZ,1


In [799]:
train_set.groupby('produt_ids')['ID'].count() 

KeyError: 'produt_ids'

> proposed feature engineering

1. 